## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [1]:
import pandas as pd
import numpy as np
import numba
import cProfile

# %load_ext line_profiler

In [2]:
# A =

In [3]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc/cnt

In [4]:
# %lprun -f f1 f1(A)

In [5]:
def f2(A):
    pass

In [6]:
def f3(A):
    acc = 0
    for ai in A:
        acc += ai
    acc += len(A) * 100
    return acc / len(A)

In [7]:
def f4(A):
    return np.mean(A)


In [8]:
@numba.njit
def f5(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc/cnt

In [9]:
# %timeit f5(A)

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.


In [10]:
df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key'] == letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [11]:
%timeit g(df)

535 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def g_my(df):
    letters = ('a', 'b', 'c', 'd', 'e')
    first_five = []
    for letter in letters:
        q = df[df['key'] == letter]
        first_five.append(q)
    return pd.concat(first_five, axis=0)

In [13]:
%timeit g_my(df)

529 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# cProfile.run(g_my(df))

## Лабораторная работа 3

In [15]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [18]:
recipes = pd.read_csv("D:\Programming\Python\BigDataPT-BDPT-\Lesson3\data\data\\recipes_sample.csv", parse_dates=["submitted"])
reviews = pd.read_csv("D:\Programming\Python\BigDataPT-BDPT-\Lesson3\data\data\\reviews_sample.csv", parse_dates=["date"], index_col=0)

In [19]:
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [20]:
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [ ]:
tenYear_reviews = reviews[reviews["date"].dt.year == 2010]

In [ ]:
tenYear_reviews["date"].iterrows()

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
